# scratch
## This notebook is for working out temporary issues and as a practice / experimentation repl
### Imports for reading in accounts

In [1]:
from finance_model.chart_of_accounts import ChartOfAccounts, unpickle_accounts
import pandas as pd
from config import config

loading environment variables
cwd = D:\Coding\Finance-Model\notebooks
inside of notebook directory
   setting new working directory: D:\Coding\Finance-Model


### unpicle accounts

In [2]:
pickle = True

if pickle:
    accounts = unpickle_accounts()
else:
    accounts = ChartOfAccounts()
    accounts.account_map
    accounts.trial_balances

Total duration for function <unpickle_accounts> is 0.0012440999853424728


#### write_accounts to csv file to double check accuracy

In [3]:
write_accounts = False
if write_accounts:
    accounts.write_accounts()

i## Setup for manipulation of the trial balance dataframe
### imports and copy the trial_lanances

In [4]:
from finance_model.finance_enums import AccountLevel, CATEGORY
tb = accounts.trial_balances.copy()

---
### Group by year

In [5]:
yearly = True
# yearly = False

if yearly:
    tb['year'] = [date[:4] for date in tb.index]
    grp = tb.groupby('year')
    # Choose the last of each grouping. This works because the data is cumulative
    tb = grp.tail(1).iloc[:, :-1]

### Merge with detailed map

In [6]:
tb = tb.T

tb[AccountLevel.ACCOUNT_NO] = tb.index
tb = tb.merge(accounts.detailed_account_map, how='left', on=AccountLevel.ACCOUNT_NO)

In [30]:
grp = tb.groupby(AccountLevel.CATEGORY)
grp.first(2)
grp.get_group(CATEGORY.ASSETS)
a = grp.sum().iloc[:, :-6].T
# a[CATEGORY.ASSETS]

a

AccountLevel.CATEGORY,ASSETS,COGS,EQUITY,LIABILITIES,OPERATING_EXPENSES,OTHER_EXPENSES,REVENUE
2017-12-31,-972617.21,-4225229.11,-291698.04,154798.30,-2311968.51,-5824.66,7652539.23
2018-12-31,-1003344.35,-4356327.82,-165181.09,171115.41,-2380507.70,-7895.31,7742140.86
2019-12-31,-1059030.32,-4795027.36,-355080.17,169558.80,-2496942.19,-2783.51,8539304.75
2020-12-31,-1373599.39,-3915939.74,-678217.69,138346.55,-2511344.51,-7278.73,8348033.51
2021-12-31,-1495487.67,-3447824.77,-1165383.89,119800.04,-1737638.27,-4805.94,7731340.50
2022-12-31,-1652634.14,-5236419.23,375602.10,119119.96,-2689231.92,-6002.26,9089565.49
2023-08-31,-2362932.61,-4359960.12,783514.18,566241.94,-1549829.80,-3658.75,6926625.16


In [34]:
(a[CATEGORY.REVENUE] + a[CATEGORY.COGS] + a[CATEGORY.OPERATING_EXPENSES] + a[CATEGORY.OTHER_EXPENSES])/a[CATEGORY.REVENUE]
# grp.get_group(CATEGORY.LIABILITIES)

2017-12-31    0.144987
2018-12-31    0.128829
2019-12-31    0.145744
2020-12-31    0.229212
2021-12-31    0.328672
2022-12-31    0.127389
2023-08-31    0.146273
dtype: float64

In [36]:
grp2 = tb.groupby([AccountLevel.CATEGORY, AccountLevel.SUBCATEGORY])
grp2
# grp.get_group(CATEGORY.EQUITY)

In [26]:
for i in CATEGORY:
    print(f'i = {i}')

i = ASSETS
i = LIABILITIES
i = EQUITY
i = REVENUE
i = COGS
i = OPERATING_EXPENSES
i = OTHER_EXPENSES
i = OTHER_INCOME


## Filter transposed trial balances
### Filter by one category

In [ ]:
a = tb.loc[:,AccountLevel.CATEGORY:AccountLevel.ACCOUNT] == CATEGORY.ASSETS
b= a.sum(axis=1) == 1
tb[b]

In [ ]:
tb

In [ ]:
len(tb)

In [ ]:
b == 1

In [8]:
len(tb.columns)

119

---
# Groupby practice

In [26]:
food_data = {
          "Item": ["Banana", "Cucumber", "Orange", "Tomato", "Watermelon"],
          "Type": ["Fruit", "Vegetable", "Fruit", "Vegetable", "Fruit"],
          "Price": [0.99, 1.25, 0.25, 0.33, 3.00]
        }
 
supermarket = pd.DataFrame(data = food_data)

supermarket

,Item,Type,Price
0,Banana,Fruit,0.99
1,Cucumber,Vegetable,1.25
2,Orange,Fruit,0.25
3,Tomato,Vegetable,0.33
4,Watermelon,Fruit,3.00


In [32]:
groups = supermarket.groupby("Type")
groups

In [39]:
groups["Price"].mean()

Type
Fruit        1.413333
Vegetable    0.790000
Name: Price, dtype: float64

In [30]:
groups.get_group("Vegetable")

,Item,Type,Price
1,Cucumber,Vegetable,1.25
3,Tomato,Vegetable,0.33
